<h2 align='center'> Stainless Steel Classification </h3>

<p> The General idea of this notebook is to highlight all the process and subprocesses done for Stainless Steel Classification.<br>For this project we have used the dataset provided <a href="https://www.kaggle.com/code/kerneler/starter-tig-stainless-steel-304-ee6df282-0/data">Here</a>. </p>

<h4> <u> Dataset Discussion </u> </h4>
<div>
    <img src='Assets\Dataset_Labels.png' align='right' width=600 height=300>
    <p align='left'><br>Our Dataset has been label encoded as following: <br> The idea is that provided an image our classifier will be able to classify the results as one of the following labels. <br><br>["good weld","burn through","contamination","lack of fusion","lack of shielding gas","high travel speed"]<br><br>
    Since we are following supervised approach we have a json file which maps each image with corresponding output labels which can be used to map each image with corresponding labels. <br><br> The mentioned json file can be seen below: <br> <br></p>
    <img src='Assets\json_files.png' width=550>
</div>


<h3> 0. Intalling Dependencies </h3>

In [ ]:
!pip install tensorflow 
!pip install -U scikit-learn
!pip install numpy
!pip install pillow
!pip install matplotlib

<h3> 1. Importing Dependencies </h3>

Let's import all the dependencies so we can use it later

In [ ]:
import json
import numpy as np 
from PIL import Image
import tensorflow as tf
import os
import json
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)

<h3> 1. PreProcessing </h3>

<p> Once Downloaded please unzip the dataset on the same working directory and the folder directory should be: <br>

<i>../ss304/train/train.json</i>
</p>


In [93]:
def convert_to_array(i,features,f):

    if f==0:
        image=features[i]
        
    else: 
        image=i
    
    image = Image.open(image)
    image=image.resize((200,200))
    numpydata = np.asarray(image)
    
    return numpydata


def get_images_array_and_labels(path):
    print('Getting all the images, Resizing it and converting it to numpy array so that it can be fitted on CNN. It might take some time depending upon processing speed')
    with open(path,'r') as datasets:
        datasets=json.load(datasets)
    features=list(datasets.keys())
    features=['/'.join(path.split('\\')[0:-1])+'/'+str(i) for i in features]
    features=np.array([convert_to_array(i,features,0) for i in range(len(features))])
    labels=np.array(list(datasets.values())).reshape(-1,1)
    #labels=tf.keras.utils.to_categorical(labels)
    return features,labels
    
    

X,Y=get_images_array_and_labels('ss304\\train\\train.json')
Xtest,Ytest=get_images_array_and_labels('ss304\\test\\test.json')

Getting all the images, Resizing it and converting it to numpy array so that it can be fitted on CNN. It might take some time depending upon processing speed


KeyboardInterrupt: 

In [ ]:
def write_training_history(history):
    with open('training_checkpoints.json','w') as f:
            json.dump(history,f)

def load_training_history():
    with open('training_checkpoints.json','r') as f:
            f=json.load(f)
    return f


def get_updated_history(history):
    
    if 'training_checkpoints.json' not in os.listdir(os.getcwd()):
        write_training_history(history)
    else: 
        f=load_training_history()
        f['loss']=f['loss']+history['loss']
        f['accuracy']=f['accuracy']+history['accuracy']
        f['val_loss']=f['val_loss']+history['val_loss']
        f['val_accuracy']=f['val_accuracy']+history['val_accuracy']
        write_training_history(f)
    
    return load_training_history()

def load_previous_model(CNN_model):
    if 'Stainless_Steel_Classification_model.h5' in os.listdir(os.getcwd()):
        CNN_model=tf.keras.models.load_model('Stainless_Steel_Classification_model.h5')
    else: 
         if 'training_checkpoints.json' in os.listdir(os.getcwd()):
             os.remove('training_checkpoints.json') 
    
    return CNN_model

def plot_metrics_for_training(history):
    
    plt.subplot(221)
    plt.plot(history['accuracy'], label='accuracy')
    plt.plot(history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.001, 1.25])
    plt.legend(loc='lower right')

    plt.subplot(222)
    plt.plot(history['loss'], label='loss')
    plt.plot(history['val_loss'], label = 'val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    plt.ylim([0.001, 4.5])
    plt.legend(loc='lower right')


def create_cnn_model():
    model=tf.keras.models.Sequential()

    #Convlution Process Starts here.
    model.add(tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(200, 200, 1)))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #Fully Connected Layer from here 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001)))
    model.add(tf.keras.layers.Dense(16,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001)))
    model.add(tf.keras.layers.Dense(6,activation='softmax',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))) 
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    #Compile the CNN Model
    model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
    model.summary()
    return model
    

CNN_model=create_cnn_model()

In [ ]:
CNN_model=load_previous_model(CNN_model)
history_for_plotting=CNN_model.fit(X,Y,epochs=100,batch_size=64,validation_data=(Xtest,Ytest),shuffle=True)
history_for_plotting=get_updated_history(history_for_plotting.history)
CNN_model.save('Stainless_Steel_Classification_model.h5')


In [ ]:
plot_metrics_for_training(history_for_plotting)

<h3> [Last Step No] Inference Testing </h3>

In [ ]:
#var1='ss304/train/160705-114214-50mmLens w.s.75cm.m/image-0001.png' #op=5,high travel speed
#var2='ss304/test/160705-121434-50mmLens added slug/image-0001.png'  #op=2,contaimination
#var3='ss304/valid/160705-114925-50mmLens w.s.16cm.m/image-0150.png' #op=0,good weld 
#var4='ss304/test/160708-145105-50mmLens 350A w.s.Lev16 g.f.20L.m try joining 10mm Plate/image-0042.png' #op=3, lack of fusion


CNN_model=load_previous_model(CNN_model)
var_input=input('Enter path of the image to test: ')
op_list=["good weld","burn through","contamination","lack of fusion","lack of shielding gas","high travel speed"]
test_image=convert_to_array(var_input,[],1)
prediction=CNN_model.predict(test_image.reshape(1,200,200,1))
print(prediction)
prediction=np.argmax(prediction)
print(prediction)
print(op_list[prediction])